# **Experimento com RNN Bidirecional: Verificar saídas e estados do LSTM e GRU para diferentes argumentos de entrada**

# **Bidirectional Recurrent Neural Networks (RNN) applied to Natural Language Processing (NLP)**

- Neste projeto, vamos **avaliar os efeitos relacionados à utilização de diferentes argumentos ao se invocar o GRU ou LSTM**.
- Em particular, podemos passar como argumento `return_sequences = True`, que fará o sistema retornar as saídas de cada ponto no tempo, ao invés de apenas a última saída.
- Também podemos passar `return_states = True`, o que fará o sistema retornar os estados das camadas ocultas (hidden states) em adição à saída da rede.
- Ao se trabalhar com uma API Machine Learning, uma das coisas mais importantes é compreender o formato de cada uma das entradas e saídas.

Neste projeto, **vamos provar que, dependendo da unidade recursiva utilizada, o hidden state será exatamente igual à saída** (`hidden state = output`).
- O diferencial deste projeto em relação ao da RNN (saídas LSTM e GRU) **será a utilização da unidade bidirecional.** 
- O método bidirecional avalia tanto a saída da unidade recursiva padrão (sequência e hidden states calculados do 1º ao último elemento da sequência); quanto avalia a unidade em ordem reversa (saída e hidden states quando a sequência é avaliado do último para o primeiro elemento). 

## **Adaptação da RNN para o caso bidirecional**

Poucas mudanças são necessárias no código. Basicamente, devemos adicionar a "wrapper layer" (camada de empacotamento).

- Para isso, simplesmente substituímos:

```
LSTM (M)
```
Por:


```
Bidirectional (LSTM(M))
```
- Isto converte automaticamente a RNN em uma RNN bidirecional.

# **Quando não utilizar uma RNN bidirecional**

Embora poderosas, existem algumas situações nas quais o uso da rede bidirecional não é recomendável (**preferível utilizar a RNN convencional**):
- **RNNs bidirecionais não devem ser empregadas quando se deseja prever comportamento ou valor futuro**.
- Não há sentido em utilizar inputs que estão ainda mais distantes no futuro (os inputs da RNN reversa). Isto iria requerer dados que ainda não existem, justamente os que desejamos prever.

- Embora este seja um problema para análise de séries temporais com RNNs, não costuma ser uma limitação em NLP. Isso porque, em NLP, costumamos usar a sequência completa (a frase, trecho, ou texto) como input de uma só vez.

# **Importar bibliotecas para análise**

In [1]:
# https://deeplearningcourses.com/c/deep-learning-advanced-nlp
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Bidirectional
import numpy as np
import matplotlib.pyplot as plt

try:
  import keras.backend as K
  if len(K.tensorflow_backend._get_available_gpus()) > 0:
    from keras.layers import CuDNNLSTM as LSTM
    from keras.layers import CuDNNGRU as GRU
except:
  pass

# **Configurações de parâmetros de forma dos dados**

```
- T = sequence length (number of words);
- D = input dimensionality (size of the word vectors);
- M = hidden layer size;
- K = number of output classes.
```



In [2]:
T = 8
D = 2
M = 3

Ao utilizar valores diferentes para cada um destes parâmetros, se torna mais fácil localizá-los e acompanhar seus fluxos (tracking) pelo código.

Vamos criar também dados de entrada aleatórios.
- Esta amostra será uma matriz de números aleatórios entre -1 a 1, de dimensão T x D, criada com a correspondente função do NumPy.

In [3]:
X = np.random.randn(1, T, D)
print(X)

[[[-0.49577208  0.63115627]
  [ 2.21774981 -1.82282658]
  [-0.44690162 -2.15102235]
  [ 0.35742117  0.18057583]
  [ 0.37160003  0.59970863]
  [ 2.15739408 -0.8071703 ]
  [-2.86387403 -0.82350849]
  [ 1.14254     0.16048017]]]


Esta matriz poderia ser a **representação de uma única sequência de vetores de palavras, ou poderia ser algum outro sinal obtido**.
- Note que ela será a **camada de input fornecida ao código Keras**.

# **Utilizar os inputs como argumento da RNN bidirecional**

Vamos realizar dois experimentos e, em ambos, forneceremos o argumento `return_states = True` para que o sistema retorne todos os valores de saída e hidden states, e não apenas o final.

1) O que é retornado se `return_sequences = True`?

2) O que é retornado se `return_sequences = False`?

# Definindo-se `return_sequences = True`

In [6]:
input_ = Input(shape=(T, D))
rnn = Bidirectional(LSTM(M, return_state=True, return_sequences=True))
# rnn = Bidirectional(LSTM(M, return_state=True, return_sequences=False))
x = rnn(input_)

A seguir, nós chamaremos: 
- A saída o: "output"; 
- h1 e c1: "hidden state" e "cell state" para o LSTM calculado na ordem direta (primeiro em direção ao último elemento da série ou sequência); e
- h2 e c2: "hidden state" e "cell state" para o LSTM calculado na ordem reversa (do último para o primeiro elemento da série).

NOTA: caso `model.predict` fosse igualado a menos parâmetros (por exemplo, apenas a o, h, e c), seria retornada uma exceção.
- **Surgiria uma mensagem de erro** informando que o método `.predict` retornou 5 valores ou objetos, mas foram fornecidos apenas 3.
- Assim, caso não se iguale o lado esquerdo (em termos de quantidade de objetos) ao lado direito (em termos de objetos ou valores retornados pelo método), será mostrada uma mensagem de erro.

In [7]:
model = Model(inputs=input_, outputs=x)
o, h1, c1, h2, c2 = model.predict(X)
print("o:", o)
print("o.shape:", o.shape)
print("h1:", h1)
print("c1:", c1)
print("h2:", h2)
print("c2:", c2)

o: [[[ 9.70592443e-03  4.48420420e-02 -1.12583913e-01 -9.79669318e-02
   -7.22687095e-02 -1.80552602e-01]
  [ 2.84550954e-02 -3.54103707e-02  8.71853977e-02 -3.15242976e-01
    7.23955110e-02 -6.09104812e-01]
  [-2.11024851e-01  6.58223927e-02  4.92125332e-01 -1.77336514e-01
    7.97866210e-02 -3.60927880e-01]
  [-5.59206754e-02 -4.80616987e-02  2.32282817e-01  1.61029175e-02
   -1.03880040e-01 -8.41613114e-02]
  [ 3.90495360e-02 -7.91260526e-02  1.36370406e-01  3.18825394e-02
   -7.15364739e-02 -1.12178780e-01]
  [ 2.33535677e-01 -1.10345371e-01  1.83673233e-01 -1.51308104e-02
    9.61242244e-02 -3.42678905e-01]
  [-1.12969913e-02  4.26956713e-01  8.50621611e-02  1.14945441e-01
    6.52327538e-02 -1.13008726e-04]
  [ 1.11003652e-01  6.54969141e-02  4.44225147e-02 -4.97316942e-02
   -7.19316751e-02 -4.48318906e-02]]]
o.shape: (1, 8, 6)
h1: [[0.11100365 0.06549691 0.04442251]]
c1: [[0.17323905 0.20996505 0.14240615]]
h2: [[-0.09796693 -0.07226871 -0.1805526 ]]
c2: [[-0.20612021 -0.18337

Note que o comando `model.predict(X)` calcula os valores previstos pelo modelo `model` para cada um dos valores do dataframe `X` fornecido como input.

`o.shape: (1, 8, 6)` indica que o é um array/matriz linha (dimensão igual a 1) formado por 8 arrays de 6 elementos.
- Note que todos os elementos dos arrays que formam o estão multiplicados por potências de 10: e-01 indica o fator x 0.1 = 10^(-1) e e-02 indica o fator x 0.01 = 10^(-2).
- Do experimento anterior, **lembramos que o "hidden state" deve corresponder à última saída, já que a saída contém simplesmente os "hidden states" a cada passo temporal** (cada valor temporal da série).

Note **o último array de o**:

```
[ 1.11003652e-01  6.54969141e-02  4.44225147e-02 -4.97316942e-02
   -7.19316751e-02 -4.48318906e-02]
```
Os **3 primeiros elementos deste array representam h1**:

```
h1: [[0.11100365 0.06549691 0.04442251]]
```
Ou seja, a última saída da série avaliada em ordem direta, do primeiro ao último elemento, é realmente h1.



Repare agora no **primeiro array de o**:

```
[ 9.70592443e-03  4.48420420e-02 -1.12583913e-01 -9.79669318e-02
   -7.22687095e-02 -1.80552602e-01]
```
Veja que **os 3 últimos elementos correspondendem a h2**:

```
h2: [[-0.09796693 -0.07226871 -0.1805526 ]]
```
De fato, h2 é o hidden state final (saída) que é obtido para a sequência avaliada em ordem inversa, ou seja, do último para o primeiro elemento.

- Assim, **h1 será mostrado como os 3 primeiros elementos** (pois fornecemos a dimensão 3 no início deste notebook) do último array (saída da ordem direta); **enquanto h2 será mostrado como os 3 últimos elementos do primeiro array** (saída da ordem reversa).
- Repare que isso corresponde à operação de concatenar os vetores direto e reverso em uma única saída, ou seja, em juntá-los em um único array de arrays.

Nas configurações de formato dos dados, fornecemos `M = 3`, razão pela qual cada hidden state é representado por 3 elementos.



# Definindo-se `return_sequences = False`

Note que apenas desmarcamos uma das linhas que estava indicada como comentário (`#`); e marcamos a outra linha como comentário.
- Assim, preservamos o código, indicando que mudança foi aplicada.
- No caso, apenas o valor do argumento `return_sequences` apresenta diferenças entre uma linha e a outra.
- Isto fará com que a saída o seja representada por um único array, e não mais por um array de arrays. 
- **Apenas os hidden states de saída da ordem direta, e os hidden states de saída da ordem reversa serão apresentados** (os demais hidden states serão omitidos).

In [8]:
input_ = Input(shape=(T, D))
# rnn = Bidirectional(LSTM(M, return_state=True, return_sequences=True))
rnn = Bidirectional(LSTM(M, return_state=True, return_sequences=False))
x = rnn(input_)

In [9]:
model = Model(inputs=input_, outputs=x)
o, h1, c1, h2, c2 = model.predict(X)
print("o:", o)
print("o.shape:", o.shape)
print("h1:", h1)
print("c1:", c1)
print("h2:", h2)
print("c2:", c2)

o: [[ 0.28639564  0.02829732 -0.32982442  0.23900957  0.11422075 -0.04232233]]
o.shape: (1, 6)
h1: [[ 0.28639564  0.02829732 -0.32982442]]
c1: [[ 0.5550117   0.06793963 -0.67570615]]
h2: [[ 0.23900957  0.11422075 -0.04232233]]
c2: [[ 0.43932542  0.2390522  -0.06943097]]


Note que `o.shape: (1, 6)` indica que o é um array/matriz linha (dimensão igual a 1) formado por 6 elementos.

```
o: [[ 0.28639564  0.02829732 -0.32982442  0.23900957  0.11422075 -0.04232233]]
```
Repare que os 3 primeiros elementos de o correspondem a h1 (hidden state final da ordem direta):

```
h1: [[ 0.28639564  0.02829732 -0.32982442]]
```
Já os 3 últimos elementos correspondem a h2 (hidden state final da ordem reversa):

```
h2: [[ 0.23900957  0.11422075 -0.04232233]]
```

Assim, quando definimos `return_sequences = False`, o array de saída será formado pelos elementos que são efetivamente os valores de saída de h1 e h2.
- Repare no teste anterior que os valores correspondentes a h1 estariam no primeiro array, caso fosse definido `return_sequences = True`.
- Desta forma, **a saída aqui não é simplesmente o último elemento da sequência formada**, mas sim a primeira parte do último elemento (último array) concatenada à segunda parte do primeiro array.
- Assim, quando `return_sequences = False`, a saída é efetivamente formada pela concatenação dos arrays h1 e h2 mostrados.




# **Conclusão**

- O objetivo deste notebook é ilustrar as diferentes saídas ("output", o) e estados ("hidden state", h, e "cell state", c) retornados pelo sistema quando definimos `return_state = True` ou `return_sequences = True`.
- Em notebooks mais complexos sobre RNN bidirecional, volte aqui para recordar as saídas e estados esperados, bem como as respectivas interpretações.
- Uma técnica útil para o "debug" da rede neural é verificar se tudo está no formato/dimensão correta a cada etapa de cálculo.
- Muitas vezes, **perdemos esta granularidade ao utilizar operações de agregação como Pooling ou soma das médias**. Este caso é particularmente difícil de ser corrigido, pois o sistema retorna um valor incorreto ao invés de uma mensagem de erro, podendo passar a impressão que o código está correto quando não o está.